In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyrsm as rsm
import seaborn as sns
import statsmodels.formula.api as smf
from statsmodels.genmod.families import Binomial
from statsmodels.genmod.families.links import logit

# increase plot resolution
mpl.rcParams["figure.dpi"] = 150

THIS IS NOT A SOLUTION NOTEBOOK. DETAILS TO FOLLOW ON HOW YOU CAN SUBMIT YOUR CSV FILE WITH PROSPECTS TO TARGET

In [ ]:
bbb = pd.read_pickle("data/bbb_pre.pkl")
bbb

In [ ]:
bbb[np.isin(bbb.buyer, ["yes", "no"])].shape

In [ ]:
bbb["buyer_yes"] = rsm.ifelse(
    bbb.buyer == "yes", 1, rsm.ifelse(bbb.buyer == "no", 0, np.NaN)
)
rsm.format_nr(np.isnan(bbb.buyer_yes).sum())

In [ ]:
bbb.buyer_yes.value_counts(dropna=False)

In [ ]:
# different cost and margin information compared to previous examples
cost = 0.99
margin = 9
breakeven = cost / margin
breakeven

### What percentage of customers bought?

In [ ]:
train = bbb.training == 1
pd.crosstab(
    index=bbb.loc[train, "buyer"], columns="Response rate", normalize="columns"
).transform(lambda x: (100 * x).round(2).astype(str) + "%")

### Create quintile variables for recency, frequency and monetary.

Hint: Review the rfm-bbb.py python code file that walks through the calculations for the Bookbinders RFM analysis in detail. Use the `xtile` function from the `pyrsm` package to create the recency, frequency, and monetary value quantiles. Note the use of `rev=True` for `freq_iq` and `mon_iq` to ensure the best customers are in the 1st quantile

In [ ]:
bbb = bbb.assign(

)

### Create bar charts showing the response rate 

The proportion of customers who bought AHOF per recency, frequency, and monetary quintile (i.e., 3 plots)


In [ ]:
fig = rsm.prop_plot(bbb[train], "rec_iq", "buyer", "yes")

In [ ]:
fig = rsm.prop_plot(bbb[train], "freq_iq", "buyer", "yes")

In [ ]:
fig = rsm.prop_plot(bbb[train], "mon_iq", "buyer", "yes")

Create the `rfm_iq` index.

In [ ]:
bbb["rfm_iq"] = 

Next, create the sequential RFM variables using select `rec_iq` as the variable to group by for `freq_sq`. Select both `rec_iq` and `freq_sq` as the variables to group by for `mon_sq`. Note that we do not need to create a `rec_sq` variable as this is equivalent to `rec_iq`.

In [ ]:
bbb["freq_sq"] = 
bbb["mon_sq"] = 

Next create the `rfm_sq` index.

In [ ]:
bbb["rfm_sq"] = 

The line in the plot below shows the break-even point. Cells with a response rate above 0.0247 are predicted to be profitable.

In [ ]:
plt.figure(figsize=(16, 7))
fig = rsm.prop_plot(
    bbb[train], "rfm_iq", "buyer", "yes", breakeven=breakeven
)
fig.set_xticklabels(fig.get_xticklabels(), rotation=90)
fig = fig.set(xlabel="Independent RFM index (rsm_iq)")

In [ ]:
plt.figure(figsize=(16, 7))
fig = rsm.prop_plot(
    bbb[train], "rfm_sq", "buyer", "yes", breakeven=breakeven
)
fig.set_xticklabels(fig.get_xticklabels(), rotation=90)
fig = fig.set(xlabel="Sequential RFM index (rsm_sq)")

In [ ]:
Demonstrate ghat a logistic regression can be just as flexible as an RFM model (i.e., avoid imposing any relationship between RFM and the probability/odds of response)

In [ ]:
lr = smf.glm(
    formula="buyer_yes ~ rfm_sq",
    family=Binomial(link=logit()),
    data=bbb[train],
).fit()

In [ ]:
rsm.or_ci(lr)

In [ ]:
bbb["resp_logit"] = lr.predict(bbb)

In [ ]:
rfm_logit = bbb.groupby("rfm_sq").resp_logit.agg(np.nanmean).reset_index()
plt.figure(figsize=(16, 7))
fig = sns.barplot(x="rfm_sq", y="resp_logit", color="slateblue", data=rfm_logit)
fig.axhline(breakeven, linestyle="dashed", linewidth=1)
fig.set_xticklabels(fig.get_xticklabels(), rotation=90)
fig = fig.set(
    xlabel="Sequential RFM index (rsm_sq)",
    ylabel="Predicted probability of buyer equal to 'yes'",
)

In [ ]:
def perf_calc(
    df, mail, intro="", perf="buyer", lev="yes", cost=cost, margin=margin, prn=True
):
    """
    A function to calculate performance stats for different targeting methods

    Parameters
    ----------
    df : Pandas dataframe
    intro: str
        Text introduction to use for printed performance output
    mail: str
        Column name of the mailing variable to use
    perf: str
        Column name of the performance variable to use
    lev : str
        Level in performance variable to use
    prn : bool
        Print performance output (True or False)

    Returns
    -------
    profit : float
        Estimate of the profit from the selected targeting strategy applied to the full customer database
    ROME : float
        Estimate of theh return on marketing investment (ROME) from the selected targeting strategy applied to the full customer database
    """


    # add your code here
    
    return profit, ROME

In [ ]:
profit_nt, ROME_nt = perf_calc(
    bbb.assign(mailto_nt=True), mail="mailto_nt", intro="If bbb does not apply targeting"
)

Offering the deal only to those the customers in (sequential)

RFM cells with a response rate that is greater than the breakeven response rate. Specifically, follow these steps:
- Determine which RFM cells (using the sequential n-tiles approach) have response rates exceeding the breakeven rate
- Determine the number of customers in these profitable cells
- Determine the number of buyers in these profitable cells
- Determine the projected profit and the return on marketing expenditures

In [ ]:
def mailto(x, breakeven):
    return np.nanmean(x) > breakeven

In [ ]:
bbb["mailto_iq"] = bbb.groupby("rfm_iq").buyer_yes.transform(
    mailto, breakeven=breakeven
)

profit_iq, ROME_iq = perf_calc(bbb, mail="mailto_iq", intro="Based on independent RFM")

In [ ]:
bbb["mailto_sq"] = bbb.groupby("rfm_sq").buyer_yes.transform(
    mailto, breakeven=breakeven
)
profit_sq, ROME_sq = perf_calc(bbb, mail="mailto_sq", intro="Based on sequential RFM")

In [ ]:
bbb["mailto_logit"] = bbb.resp_logit > breakeven
profit_lr, ROME_lr = perf_calc(
    bbb, mail="mailto_logit", intro="Based on logistic regression"
)

In [ ]:
bbb["resp_rfm_iq"] = bbb.groupby("rfm_iq")["buyer_yes"].transform(np.nanmean)
bbb["resp_rfm_sq"] = bbb.groupby("rfm_sq")["buyer_yes"].transform(np.nanmean)
bbb["breakeven"] = breakeven

### Create a plot with all profit and ROME numbers

In [ ]:
dat = pd.DataFrame(
    {
        "name": ["No targeting", "Indep. RFM", "Seq. RFM", "Logit"],
        "Profit": [profit_nt, profit_iq, profit_sq, profit_lr],
        "ROME": [ROME_nt, ROME_iq, ROME_sq, ROME_lr],
    }
)

In [ ]:
plt.figure(figsize=(8, 5))
fig = sns.barplot(x="name", y="Profit", color="slateblue", data=dat)
fig.set(xlabel="", ylabel="Profit", title="Campaign profit")
for index, row in dat.iterrows():
    fig.text(
        row.name, row.Profit * 0.8, f"{row.Profit:,.0f}", ha="center", color="white"
    )

In [ ]:
plt.figure(figsize=(8, 5))
fig = sns.barplot(x="name", y="ROME", color="slateblue", data=dat)
fig.set(xlabel="", ylabel="ROME", title="Return on Marketing Expenditures (ROME)")
for index, row in dat.iterrows():
    fig.text(
        row.name,
        row.ROME * 0.8,
        f"{round((100*row.ROME), 2):,}%",
        ha="center",
        color="white",
    )

## Determine the list if prospects to target based on sequential RFM

Get the list of prospect to contact from the rest of the customer database. Your csv file should *only* contain information on acctnum for the prospect to target and nothing else

In [ ]:
to_contact = 
to_contact.acctnum.to_csv("rfm_sq_to_contact.csv", index=False)